<h1> Kafka Topology and Components </h1>


Kafka is a Publish-Subscribe messaging system used with real-time streaming data (in addition to it's ability to process batch data).

In order for Kafka to be able to handle massive volumes of data at scale coming in at rapid velocity, several components are required to setup a robust Kafka system.

As a quick reminder, below is the overall Kafka topology at a high-level:

<p align="center">

<img src= "images/Kafka_Architecture2.png">

</p>


## Main Concepts and Terminology

### Events:

> An event records the fact that "something happened" in the world or in your business. 

It's also called a record or message in the Kafka documentation. When you read or write data to Kafka, you do this in the form of events. 

Conceptually, an event has a _key_, _value_, _timestamp_, and optional _metadata headers_. Here's an example event:

`- Event key: "Alice"`

`- Event value: "Made a payment of $200 to Bob"`

`- Event timestamp: "Jun. 25, 2020 at 2:06 p.m."`

### Topics

> Events (records) are organized and durably stored in topics. Very simplified, a topic is similar to a folder in a filesystem, and the events are the files in that folder. 

An example Topic name could be "payments". Topics in Kafka are always multi-producer and multi-subscriber: a Topic can have zero, one, or many producers that write events to it, as well as zero, one, or many consumers that subscribe to these events. 

Events in a topic can be read as often as needed—unlike traditional messaging systems. Events are not deleted after consumption. Instead, the user defines for how long Kafka should retain the events through a per-topic configuration setting, after which old events will be discarded. Kafka's performance is effectively constant with respect to data size, so storing data for a long time is perfectly fine.

When a new event is published to a certain topic, it is actually appended to one of the topic's __partitions__. Events with the same event key (e.g., a customer or vehicle ID) are written to the same partition, and Kafka guarantees that any consumer of a given topic-partition will always read that partition's events in exactly the same order as they were written.

So, in summary:
- A topic is a category/feed name to which data records are stored and published.
- All Kafka data records are organized into topics. 
- Producers write data to the topics and consumers read data from the topics
- Data records plubished in the cluster remain persisted until a specified retention period has passed by.
- Each topic is divided into partitions, which contain records in an unchangeable sequence. 
- Each record within a partition is assigned and identified by a unique offset (ID)

Below is a visual representation of how these different components interact with one another:


<p align="center">
<img src= "images/Kafka Zookeeper Brokers.png" width=600>
</p>

For a video explanation on Topics, please watch the following video:

- [__Topic Introduction Video__](https://www.youtube.com/embed/kj9JH3ZdsBQ)


<p></p>

Below is a visual representation of what __partitions__ look like:

<p align="center">
<img src= "images/Kafka_Topics.png" width=600>
</p>



### Producers 

> Producers are client applications that publish (write) events to Kafka.

A Kafka producer is an application that acts as a source of data in a Kafka cluster.  For each producer, the key and value properties must be identified. Depending on the settings, a producer controls how many records it accumulates before actually transporting this data to the cluster. Producers also map each of the event messages top a topic partition, and sends a produce request to the leader of that particular partition. Using the key as an ID, events with the same key are gauranteed to go to the same topic partition.

So, in summary, a producer:
- Connects to a Kafka cluster either through zookeeper or directly to a Kafka broker
- Sends records (data) to a broker.

For a video explanation on producers, please watch the following:

- [__Producer Introduction Video__](https://www.youtube.com/embed/I7zm3on_cQQ)

### Consumers

> Consumers are applications that subscribe to (read and process) events. 

Consumers, on the other hand, are the subscribers responsible for reading records from one or more topics and one or more partitions of a topic. Suppose we  have an application that needs to read messages from a Kafka topic, run some validations against them, and write the results to another data store. In this case, the application will create a consumer object, subscribe to the appropriate topic using the provided API, and start receiving messages, validating them and writing the results. 

This may work well for a while, but what if the rate at which producers write messages to the topic exceeds the rate at which the application can validate them? If we are limited to a single consumer reading and processing the data, the application may start to break, unable to keep up with the rate of incoming messages. Obviously there is a need to scale consumption from topics. Just like multiple producers can write to the same topic, we need to allow multiple consumers to read from the same topic, splitting the data between them.

Accordingly, Kafka consumers are typically part of a __consumer group__. When multiple consumers are subscribed to a topic and belong to the same consumer group, each consumer in the group will receive messages from a different subset of the partitions in the topic. This allows the Kafka system to scale and to guarantee data delivery.

In Kafka, producers and consumers are fully decoupled and agnostic of each other, which is a key design element to achieve the high scalability that Kafka is known for. For example, producers never need to wait for consumers. Kafka provides various guarantees such as the ability to process events exactly-once.

- There are two types of Consumers:
 
        - Low-level: Enables custom partition data consuming logic.
        - High-level: Abstracts most of the details of consuming events from Kafka.  Stores the last offset read from a specific partition.

So, in summary:
- Consumers are responsible for reading records from one or more topics.
- Consumers typically work in consumer groups.
- Consumers are agnostic of producers, which provides flexibility to Kafka.
- There are two types of consumers: low-level and high-level.


For a video explanation on consumers, please watch the following:


- [__Consumer Introduction Video__](https://www.youtube.com/embed/Z9g4jMQwog0)

### Brokers

> A Broker is a Kafka node (or server) which is part of the Kafka system


- A Kafka _cluster_ is usually composed of multiple brokers
- Each broker has a unique ID
- Brokers store the topic log partitions
- Brokers handle all requests from clients (produce, consume, and metadata) and keeps data replicated within the cluster. 
- There can be one or more brokers in a cluster.

For a video explanation on brokers, please watch the following:

- [__Brokers Introduction Video__](https://www.youtube.com/embed/jHnyBSUVcOU)


A Broker can be configured and the properties can be updated in the __server.properties__ file.

__Core configurations for a broker consist of:__
1. `broker.id`
    - This is an integer that must be set as a unique value for each Broker
2. `listeners`
    - This is the address that the socket server listens on (hostname and the port)
3. `advertised.listeners`
    - This is the Hostname and port the Broker will advertise to the Producers and Consumers
4. `log.dirs`
    - This is a comma seperated list of directories under which to store the Log files
5. `num.partitions`
    -  The number of Partitions per Topic
6. `log.retention.hours`
    - The minimum age of a Log file before deletion
7. `zookeeper.connect`
    - A comma seperated host:port pairs each corresponding to a Zookeeper server


### Zookeeper

> Zookeeper is part of the Hadoop technology stack (external to the core Kafka components yet required as part of the system).  Zookeeper is also responsible for coordinating Kafka tasks with other Big Data components.

- Kafka uses Zookeeper to manage service discovery for brokers (e.g. if a new broker joins, or a broker dies etc.)
- Maintains the state of the cluster (brokers, topics, users).


Let's take a look at what zookeeper configurations look like.  These can be found in the __zookeeper.proerties__ file.

__Core configurations for zookeeper include:__
1. `dataDir`
    -   The directory where the snapshots will be stored
2. `clientPort`
    -   The Port which clients will use to connect
3. `maxClientCnxns`
    -   Enables/Disables the per-IP limit on the number of connections
4. `admin.enableServer`
    -   Enable/Disables the adminserver to avoid port conflicts


## Configuring Zookeeper

We'll continue the hands-on excercise based on the progress of prevoius lessons, so make sure that Kafka is downloaded and untarred on your local machine.

You'll need to create a new `data` folder which will store the data records.

Within the `data` folder, you'll need to create a `kafka` and `zookeeper` subfolders.  These are required in order to update the configuration files.

Go ahead and run the below commands:

In [ ]:
mkdir data
mkdir data/kafka
mkdir data/zookeeper
ls -al

Now, your output should be similar to:

![](images/kafka-data.png)

Next, we'll need to update the __zookeeper properties__ and the __Apache Kafka log__ files to point to the new data directory we just created.

To achieve this, first create a folder called `data` and two subfolders inside it called `kafka` and `zookeeper`, then update the files to point to the newly created folders.

Next, add `listeners=PLAINTEXT://127.0.0.1:9092` to the end of the server properties file to make it point to the localhost.

Listeners are how we configure Kafka and instruct it to connect to a specific broker's hostname, IP address and port number.  

## Setting up producers, consumers, brokers, and your first topic

The next step is to start the actual Kafka cluster and to set up zookeeper, the producer, consumer, broker and our very first Kafka topic.

To achieve this, first we need to run __zookeeper__.

This can be done by running the following command:

In [ ]:
# Start the zookeeper server
bash bin/zookeeper-server-start.sh config/zookeeper.properties

The __`zookeeper-server-start.sh`__ command does the following:
- Starts the zookeeper server
- Takes as an argument the location of the zookeeper properties file

To read more about the __`zookeeper-server-start.sh`__ command and its various properties, please check the official documentation in the following link:
- __[Zookeeper official documentation](https://zookeeper.apache.org/doc/r3.6.3/zookeeperTools.html)__

*Note: `zookeeper-server-start.sh` and `zkServer.sh` are essentially the same.  The naming differnce is due to the fact that the former was named by the Kafka foundation while the latter is a zookeeper terminology. 



There should be many items being downloaded which will take a few minutes.

Now, if everything works correctly you should see an output similar to the following:

![](images/kafka-zookeeper.png)


After displaying several updates, the terminal will remain open and the cursor will be blinking.  This is normal and to be expected.  We'll leave this terminal open and continue.

The next step is to run the Kafka Broker.

Open a second terminal session (and make sure to keep the zookeeper one open) and enter the following command:

In [ ]:
# Start the broker server
cd kafka_2.13-3.0.0 
bash bin/kafka-server-start.sh config/server.properties

The __`kafka-server-start.sh`__ command starts the Kafka server.

It takes the following arguments:
- Server properties file path
- Override property value (optional)
 


If all runs correctly, you should see a long output that looks similar to the following:

![](images/kafka-server.png)

Now we have both the zookeeper and Kafka server running.  This prepares us to start producing and consuming messages.

The next step is to open other terminals and to create a Kafka __topic__ which we'll use to share messages between the local producer and consumer.

There are some required parameters such as the __partition number__ and the __replication factor__ along with the __topic name__ and the __server details__.

For the time being, we'll keep things simple and limit the number of partitions to 1 and the replication factor to 1 also.

To achieve this, run the following command:

In [ ]:
# Create a new Kafka topic and set the mandatory parameters
cd kafka_2.13-3.0.0 
bash bin/kafka-topics.sh --create --topic MyFirstKafkaTopic --partitions 1 --replication-factor 1 --bootstrap-server localhost:9092

The __`kafka-topics.sh`__ command, as the name implies, is used to create and configure Kafka topics.

You can read more about __`kafka-topics.sh`__ in this link:
- __[Kafka Topic Documentation](https://jaceklaskowski.gitbooks.io/apache-kafka/content/kafka-admin-TopicCommand.html)__

It has many required and optional features which provides flexibility for data engineers.

__Required Arguments:__
- `--create`:
    -   Creates a new topic.  This is required ony the first time we are dealing with a new topic.
- `--topic`:
    -   The topic name to create, alter, describe or delete.
- `--partitions`
    -   The number of partitions for the topic being created or altered.
- `--replication-factor`:
    -   The replication factor for each partition in the topic being created.  This is mandatory if there is no default setup in the cluster itself.
- `--bootstrap-server`:
    -   The Kafka server to connect to.  Localhost:9092 is to be used in case of a local stand-alone environment.

__Important optional arguments:__
- `alter`:
    -   Alters the number of partitions, replica assignment and/or configuration for the topic
- `config`:
    -   A topic configuration override for the topic being created or altered.  Allows configurations for:
        -   Cleanup policy
        -   Compression type
        -   Delete Retention time
        -   Flushing messages
- `delete`:
    -   Deletes a topic
- `describe`:
    -   Lists the details for a particualr topic
- `list`:
    -   Lists all the available topics

Assuming the above code ran correctly, your output for this command should look something like:

![](images/kafka-topic.png)


A producer can be configured and the properties can be updated in the __producer.properties__ file.

__Core configurations for a producer consist of:__
1. `bootstrap.servers`
    - This is a list of brokers used for bootstrapping knowledge about the rest of the cluster.
2. `compression.type`
    - Allows specifying the compression codec for all data generated (none, gzip, lz4, zstd)
3. `partitioner.class`
    - Name of the partitioner class to be used for partitioning events (default is random spreading)
4. `request.timeout.ms`
    - The maximum amount of time the client will wait for the response of a request
5. `buffer.memory`
    -  The total bytes of memory the producer can use to beffer records waiting to be sent to the server


Now, we need to open two additional terminals (so the total opened terminals will be five).  One terminal will be for the producer, and the second is for the consumer.

We will immulate how Kafka operates in production.  Messages entered into the producer console will automatically arrive and be displayed in the consumer console.

To try this, open a new terminal for the __producer__ by running the following commands:

In [ ]:
# Call the Kafka producer and instruct it to connect to the specified topic
cd kafka_2.13-3.0.0/bin 
bash kafka-console-producer.sh --topic MyFirstKafkaTopic --bootstrap-server localhost:9092 

The __`kafka-console-producer.sh`__ command is used to initiate the producer from a user console and provide it with the required configuration.

__Required Arguments:__
- `topic`:
    -   Topic name to which the Producer will send the data to
- `bootstrap-server`:
    -   The Kafka server to connect to


__Important Optional Arguments:__
- `batch-size`:
    -   Number of messages to send in a single batch if they're not being sent synchronously (default is set at 200)
- `compression-code`:
    -   The data compression codec used.  Can be one of the following (default is gzip):
        - None
        - Gzip
        - Snappy
        - Lz4
        - Zstd
- `max-memory-bytes`:
    -   The total memory used by the producer to buffer records waiting to be sent to the server
- `max-partitions-memory-bytes`:
    -   The buffer memory size allocated for a partition.  When data records are received which are smaller than this size, the producer will attempt to group them together until the specified size is reached
- `property`: A mechanism to pass user-defined properties in the form of key = value to the message reader.  This allows custom configurations for a user-defined message reader

For further information about this command, check the [__Kafka Producer documentation__](https://docs.cloudera.com/runtime/7.2.0/kafka-managing/topics/kafka-manage-cli-producer.html?).


A consumer can be configured and the properties can be updated in the __consumer.properties__ file.

__Core configurations for a consumer consist of:__
1. `bootstrap.servers`
    - This is a list of brokers used for bootstrapping knowledge about the rest of the cluster.
2. `group.id`
    - The consumer group ID
3. `auto.offset.reset`
    - Tells the consumer what to do when there is no initial offset in Kafka or if the current offset does not exist anymore on the server.  Options include: latest, earliest, none

To continue with our hands-on example, the final remaining step is to call the Kafka consumer.  

To achieve this, in parallel, open a new terminal for the __consumer__ by running the following commands:

In [ ]:
# Call the Kafka consumer and instruct it to consume messages from the specified topic
cd kafka_2.13-3.0.0/bin 
bash kafka-console-consumer.sh --topic MyFirstKafkaTopic --from-beginning --bootstrap-server localhost:9092 

The __`kafka-console-consumer.sh`__ command is used to initiate the consumer, which will then read(consume) the data records from the specified topic.

__Required arguments:__
- `topic`:
    -   Topic name from which to consume the data records.
- `bootstrap-server`:
    -   The Kafka server to connect to.

__Important optional arguments:__
- `consumer-property`:
    -   A mechansim to pass user-defined properties in the form of kev=value to the consumer.
- `consumer.config`:
    -   The consumer configuration properties file.  Note that the `consumer-property` settings take precendence over this file.
- `from-beginning`
    -   Tells the consumer that if it doesn't already have an established offset to consume from, start with the earliest message present in the log.
- `max-messages`:
    -   The maximum number of messages to consumer before exiting.  If it's not set, consumption is continual.
- `offset`:
    -   A non-negative number representing the offset to consume data records from.  Can also use 'earlist' or 'latest'.  The default is 'latest'.
- `partition`:
    -   The partition to consume data records from. The default is for consumption to start from the end of the partition.

Make sure to check [__the consumer documentation__](https://docs.cloudera.com/runtime/7.2.10/kafka-managing/topics/kafka-manage-cli-consumer.html) for more details.

Now, with both window terminals side by side, click on the __producer__ terminal and enter the following JSON data:

`{
  "vehicleId":"0bf45cac-d1b8-4364-a906-980e1c2bdbcb",
  "vehicleType":"Taxi",
  "routeId":"Route-37",
  "longitude":"-95.255615",
  "latitude":"33.49808",
  "timestamp":"2017-10-16 12:31:03",
  "speed":49.0,
  "fuelLevel":38.0
}`

`{
  "vehicleId":"d5fd4b42-3742-11ec-8d3d-0242ac130003",
  "vehicleType":"Bus",
  "routeId":"Route-32",
  "longitude":"-81.615234",
  "latitude":"13.56599",
  "timestamp":"2017-10-17 14:22:03",
  "speed":37.0,
  "fuelLevel":19.0
}`

`{
  "vehicleId":"04be0177-8326-4b59-a15d-19f015c0be63",
  "vehicleType":"Passenger",
  "routeId":"Route-19",
  "longitude":"-15.611331",
  "latitude":"44.59816",
  "timestamp":"2017-10-18 09:07:08",
  "speed":75.0,
  "fuelLevel":48.0
}`

You should see the messages automatically show up on the consumer terminal similr to the below output:

![](images/kafka-producer-consumer.png)


__You've just successfully completed your first Kafka hands-on excercise__

# Data Engineer vs. DevOps in Organizations

In the real-world, it is not that common for data engineers to create an entirely new Kafka cluster from scratch.

Rather, in large organizations such as Fortune 500 companies, there are multiple teams that are responsible to setup the technology infrastructure and foundation for the Data Engineers to use.

These teams are often called "Infrastructure Operations" or "DevOps" teams.  These teams work with the senior architects and are the ones responsible for creating the technology roadmap, system design and infrastructure for the rest of the organization.  For example, they will be responsible for creating and maintaining the Hadoop cluster(s), Apache Spark environment, Kafka cluster etc. They are also responsible for backups, disaster recovery, fault tolerance and the stability of the various systems.

The Data engineering team mainly focuses on system, software, and ETL development activities.  They will mainly be users of the corporate systems by obtaining access to the various servers and tools.  The data engineering team will also generally provide the technical requirements and specifications for the enviornment (such Kafka Topic details, retention period etc.) during the system design process and report any feedback they have of the current system.  

In smaller to mid-size companies, it could be possible that a data engineer will also support by creating infrastructure components such as deploying an Apache Spark cluster and connecting it to a Kafka cluster. This is sometimes called a "DataOps" role.

What is important for the Data Engineer to understand is the __configuration properties__ that each component provides and how to edit/update those properties so we're able to use them in the Data Pipeline.   

Here is a brief description of a typical role a DevOps engineer plays in large organizations:
- __[DevOps Job Description](https://targetjobs.co.uk/careers-advice/job-descriptions/devops-engineer-job-description)__

#  Key Takeaways

- Kafka is a publish-subscribe messaging system that can be used for both batch and real-time/streaming data.
- There are 5 components required to run a Kafka cluster; namely the producer, topic, consumer, the broker server and zookeeper.
- In Kafka's terminology, a topic is akin to a folder and an event is a data record and is similar to files within that folder.
- All input and output that occurs in a Kafka cluster has to be handled by a topic, which 
- Producers are used to generate data for a Kafka topology, while consumers are used to read and process that data.
- A Kafka cluster can have many producers and consumers, and they are agnostic of each other and can be configured seperately.
- A Data engineer has some similarities with, but is not identical to a DevOps engineer in large global companies. DevOps engineers are usually responsible for creating the data infrastructure required for data engineers to use, while data engineers mainly focus on creating applications for data transformations.